#### Imports

In [1]:
import os

import tensorflow as tf
tf.config.list_physical_devices('GPU')

from keras import applications, Input, Model, optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential 
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.callbacks import EarlyStopping, TensorBoard

#### Constants

In [2]:
seed = 42
target_size = 96
batch_size = 512
epochs = 100
early_stopping_patience = 10

mrl_dataset_path = os.path.join(os.getcwd(), 'data\\MRL_eye_dataset\\eyes')
print(f'MRL eye data set path:\n{mrl_dataset_path}')

own_dataset_path = os.path.join(os.getcwd(), 'data\\own_eye_dataset\\eyes')
print(f'Own eye data set path:\n{own_dataset_path}')

tensorboard_log_dir = os.path.join(os.getcwd(), 'log_dir')
print(f'\nTensorboard log path:\n{tensorboard_log_dir}')

custom_model_path = os.path.join(os.getcwd(), 'custom\\open_closed_eyes-001')
print(f'\nCustom model path:\n{custom_model_path}')

MRL eye data set path:
t:\498-dl\modules\project\implementation\model-training\closed-eyes-detection\data\MRL_eye_dataset\eyes
Own eye data set path:
t:\498-dl\modules\project\implementation\model-training\closed-eyes-detection\data\own_eye_dataset\eyes

Tensorboard log path:
t:\498-dl\modules\project\implementation\model-training\closed-eyes-detection\log_dir

Custom model path:
t:\498-dl\modules\project\implementation\model-training\closed-eyes-detection\custom\open_closed_eyes-001


### 1. MRL Eye Data Set

#### Data

In [3]:
# augmented images
train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.1,
        horizontal_flip=True,
        rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.1)

train_generator = train_datagen.flow_from_directory(
        # represents 80 percent of the data set
        os.path.join(mrl_dataset_path, 'train'),
        target_size=(target_size, target_size),
        batch_size=batch_size,
        class_mode='binary',
        seed=seed)

# unchanged images
valid_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = valid_datagen.flow_from_directory(
        # represents 20 percent of the data set
        os.path.join(mrl_dataset_path, 'validate'),
        target_size=(target_size, target_size),
        batch_size=batch_size,
        class_mode='binary')

Found 3172 images belonging to 2 classes.
Found 954 images belonging to 2 classes.


#### Model Building

In [4]:
# Using feature extractors from MobileNet
base_model =  applications.MobileNet(include_top = True , weights = 'imagenet', input_tensor = Input(shape = (target_size, target_size, 3)))
for models in base_model.layers:
  models.trainable= False
base_model = Model(inputs = base_model.input, outputs = base_model.layers[-2].output)

# Build model using MobileNet feature extractors, 2 fully connected layers, and a softmax output
model = Sequential()
for layer in base_model.layers:
  model.add(layer)
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

# compile model
model.compile(optimizer=optimizers.Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#### Training

In [5]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = early_stopping_patience, restore_best_weights = True, verbose = 1)
tensorboard_callback = TensorBoard(log_dir = tensorboard_log_dir, histogram_freq = 1)

history = model.fit(
    train_generator,
    steps_per_epoch=1000//batch_size,
    epochs=epochs,
    validation_data = validation_generator,
    validation_steps=800//batch_size,
    callbacks=[early_stopping_callback, tensorboard_callback])

# %tensorboard --logdir log_dir

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 1.4692 - accuracy: 0.4805 - val_loss: 0.8100 - val_accuracy: 0.6309
Epoch 2/100
1/1 [==============================] - 1s 1s/step - loss: 0.8587 - accuracy: 0.5781 - val_loss: 0.4089 - val_accuracy: 0.8066
Epoch 3/100
1/1 [==============================] - 1s 934ms/step - loss: 0.5182 - accuracy: 0.7500 - val_loss: 0.3185 - val_accuracy: 0.8516
Epoch 4/100
1/1 [==============================] - 1s 1s/step - loss: 0.4112 - accuracy: 0.8320 - val_loss: 0.3440 - val_accuracy: 0.8652
Epoch 5/100
1/1 [==============================] - 1s 1s/step - loss: 0.3655 - accuracy: 0.8301 - val_loss: 0.3666 - val_accuracy: 0.8613
Epoch 6/100
1/1 [==============================] - 1s 1s/step - loss: 0.3180 - accuracy: 0.8730 - val_loss: 0.3303 - val_accuracy: 0.8926
Epoch 7/100
1/1 [==============================] - 1s 1s/step - loss: 0.3345 - accuracy: 0.8594 - val_loss: 0.2749 - val_accuracy: 0.9082
Epoch 8/100
1/1 [==============

#### Evaluation

Evaluation using unseen test set representing 15% of the MRL eye data set.

In [6]:
mrl_test_datagen = ImageDataGenerator(rescale=1./255)

mrl_test_generator = mrl_test_datagen.flow_from_directory(
        os.path.join(mrl_dataset_path, 'test'),
        target_size=(target_size, target_size),
        batch_size=batch_size,
        class_mode='binary')

model.evaluate(mrl_test_generator)

Found 720 images belonging to 2 classes.
2/2 [==============================] - 0s 46ms/step - loss: 0.1554 - accuracy: 0.9292


[0.15540042519569397, 0.9291666746139526]

Evaluation using the targeted data (recorded video from the system) using the unseen test set representing 15% of the own eye data set.

In [7]:
own_test_datagen = ImageDataGenerator(rescale=1./255)

own_test_generator = own_test_datagen.flow_from_directory(
        os.path.join(own_dataset_path, 'test'),
        target_size=(target_size, target_size),
        batch_size=batch_size,
        class_mode='binary')

model.evaluate(own_test_generator)

Found 470 images belonging to 2 classes.
1/1 [==============================] - 0s 177ms/step - loss: 1.8924 - accuracy: 0.4830


[1.8924071788787842, 0.4829787313938141]

##### >>> Result: Data sets are incompatible (Low-Light IR vs. (A)BGR capture).

This most likely explains the observed weak performance of the pretrained OpenVINO model on the recorded data.

### 2. Own Eye Data Set

### 2.1 Model 1

#### Data

In [8]:
# augmented images
train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.1,
        horizontal_flip=True,
        rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.1)

train_generator = train_datagen.flow_from_directory(
        # represents 80 percent of the data set
        os.path.join(own_dataset_path, 'train'),
        target_size=(target_size, target_size),
        batch_size=batch_size,
        class_mode='binary')

# unchanged images
valid_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = valid_datagen.flow_from_directory(
        # represents 20 percent of the data set
        os.path.join(own_dataset_path, 'validate'),
        target_size=(target_size, target_size),
        batch_size=batch_size,
        class_mode='binary')

Found 2092 images belonging to 2 classes.
Found 630 images belonging to 2 classes.


#### Model Building

Model will use the feature extractors from MobileNet and add two fully connected dense layers on top.

In [9]:
# Using feature extractors from MobileNet
base_model =  applications.MobileNet(include_top = True , weights = 'imagenet', input_tensor = Input(shape = (target_size, target_size, 3)))
for models in base_model.layers:
  models.trainable= False
base_model = Model(inputs = base_model.input, outputs = base_model.layers[-2].output)

# Build model using MobileNet feature extractors, 2 fully connected layers, and a softmax output
model = Sequential()
for layer in base_model.layers:
  model.add(layer)
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

# compile model
model.compile(optimizer=optimizers.Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#### Training

In [10]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = early_stopping_patience, restore_best_weights = True, verbose = 1)
tensorboard_callback = TensorBoard(log_dir = tensorboard_log_dir, histogram_freq = 1)

history = model.fit(
    train_generator,
    steps_per_epoch=1000//batch_size,
    epochs=epochs,
    validation_data = validation_generator,
    validation_steps=800//batch_size,
    callbacks=[early_stopping_callback, tensorboard_callback])

# enable to add logs for tensorboard
# %tensorboard --logdir log_dir

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 1.6868 - accuracy: 0.4091 - val_loss: 0.6022 - val_accuracy: 0.7207
Epoch 2/100
1/1 [==============================] - 1s 1s/step - loss: 0.5505 - accuracy: 0.7480 - val_loss: 0.5748 - val_accuracy: 0.7637
Epoch 3/100
1/1 [==============================] - 1s 803ms/step - loss: 0.8000 - accuracy: 0.6591 - val_loss: 0.4364 - val_accuracy: 0.7637
Epoch 4/100
1/1 [==============================] - 1s 1s/step - loss: 0.3954 - accuracy: 0.7461 - val_loss: 0.2547 - val_accuracy: 0.8711
Epoch 5/100
1/1 [==============================] - 1s 1s/step - loss: 0.2169 - accuracy: 0.8887 - val_loss: 0.1790 - val_accuracy: 0.9355
Epoch 6/100
1/1 [==============================] - 1s 1s/step - loss: 0.1554 - accuracy: 0.9570 - val_loss: 0.1577 - val_accuracy: 0.9336
Epoch 7/100
1/1 [==============================] - 1s 1s/step - loss: 0.1535 - accuracy: 0.9297 - val_loss: 0.1482 - val_accuracy: 0.9375
Epoch 8/100
1/1 [==============

#### Evaluation

Evaluation using the targeted data (recorded video from the system) using the unseen test set representing 15% of the own eye data set.

In [11]:
own_test_datagen = ImageDataGenerator(rescale=1./255)

own_test_generator = own_test_datagen.flow_from_directory(
        os.path.join(own_dataset_path, 'test'),
        target_size=(target_size, target_size),
        batch_size=batch_size,
        class_mode='binary')

model.evaluate(own_test_generator)

Found 470 images belonging to 2 classes.
1/1 [==============================] - 0s 121ms/step - loss: 0.0737 - accuracy: 0.9787


[0.0736980140209198, 0.978723406791687]

##### >>> Result: Accuracy should be sufficient for target data, if samples are representative.<br><br>

### Export

Export the model to the distracted driver detection project for conversion to openVINO intermediate format (IR).

In [12]:
custom_model_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                                 'distracted-driving-detection\\models\\custom\\open-closed-eyes-001')
print(f'\nCustom model path:\n{custom_model_path}')

#
# NOTE: uncomment to export model
#
# model.save(custom_model_path)


Custom model path:
t:\498-dl\modules\project\implementation\distracted-driving-detection\models\custom\open-closed-eyes-001


### 2.2 Model 2

In order to reduce complexity and thus inference time a second model is built.

#### Data

The same data is used for training model 2 as for model 1.

#### Model Building

In [13]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(target_size, target_size, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.05))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.05))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.05))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(64, activation='relu'))
model.add(Dense(2,activation='softmax'))

model.compile(optimizer=optimizers.Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#### Training

In [14]:
early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = early_stopping_patience, restore_best_weights = True, verbose = 1)
tensorboard_callback = TensorBoard(log_dir = tensorboard_log_dir, histogram_freq = 1)

history = model.fit(
    train_generator,
    steps_per_epoch=1000//batch_size,
    epochs=125,
    validation_data = validation_generator,
    validation_steps=800//batch_size,
    callbacks=[early_stopping_callback, tensorboard_callback])

# %tensorboard --logdir log_dir

Epoch 1/125
1/1 [==============================] - 2s 2s/step - loss: 0.7654 - accuracy: 0.1992 - val_loss: 0.6014 - val_accuracy: 0.7617
Epoch 2/125
1/1 [==============================] - 1s 780ms/step - loss: 0.5736 - accuracy: 0.7578 - val_loss: 0.5360 - val_accuracy: 0.7422
Epoch 3/125
1/1 [==============================] - 1s 791ms/step - loss: 0.5309 - accuracy: 0.7656 - val_loss: 0.5159 - val_accuracy: 0.7578
Epoch 4/125
1/1 [==============================] - 1s 786ms/step - loss: 0.5422 - accuracy: 0.7598 - val_loss: 0.5280 - val_accuracy: 0.7734
Epoch 5/125
1/1 [==============================] - 1s 825ms/step - loss: 0.4926 - accuracy: 0.7930 - val_loss: 0.5440 - val_accuracy: 0.7598
Epoch 6/125
1/1 [==============================] - 1s 758ms/step - loss: 0.5158 - accuracy: 0.7734 - val_loss: 0.5316 - val_accuracy: 0.7598
Epoch 7/125
1/1 [==============================] - 1s 797ms/step - loss: 0.5220 - accuracy: 0.7598 - val_loss: 0.5039 - val_accuracy: 0.7754
Epoch 8/125
1/1 

### Evaluation

In [15]:
own_test_datagen = ImageDataGenerator(rescale=1./255)

own_test_generator = own_test_datagen.flow_from_directory(
        os.path.join(own_dataset_path, 'test'),
        target_size=(target_size, target_size),
        batch_size=batch_size,
        class_mode='binary')

model.evaluate(own_test_generator)

Found 470 images belonging to 2 classes.
1/1 [==============================] - 1s 505ms/step - loss: 0.5170 - accuracy: 0.7660


[0.5170083045959473, 0.7659574747085571]

#### Export

In [16]:
custom_model_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                                 'distracted-driving-detection\\models\\custom\\open-closed-eyes-002')
print(f'\nCustom model path:\n{custom_model_path}')

#
# NOTE: uncomment to export model
#
# model.save(custom_model_path)


Custom model path:
t:\498-dl\modules\project\implementation\distracted-driving-detection\models\custom\open-closed-eyes-002


#### NOTE:

If none of the models performs well in the pipeline then most likely the data is lacking variability and more data needs to be acquired.<br>Due to project time constraints, however, new/more data can most likely not be acquired for now.